In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import current_date, lit

In [0]:
df_hospedes = (
    spark.read.table("production.raw.raw_tb_hospedes")
    .select(
        F.col("hospede_id"),
        F.col("nome_registro"),
        F.col("nome_social"),
        F.col("cpf"),
        F.col("data_nascimento"),
        F.col("sexo_atribuido_nascimento"),
        F.col("identidade_genero"),
        F.col("pronome"),
        F.col("orientacao_sexual"),
        F.col("nacionalidade"),
        F.col("tipo_cliente"),
        F.col("email"),
        F.col("telefone"),
        F.col("estado"),
        F.col("cidade"),
        F.col("cep")
    )
    .withColumn("data_nascimento", F.to_date(F.col("data_nascimento")))
    .withColumn("idade", (F.datediff(F.current_date(), F.col("data_nascimento")) / 365).cast("integer"))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_hospedes.limit(5))

In [0]:
df_geolocalizacao = (
    spark.read.table("production.raw.raw_geolocalizacao_ceps")
    .select(
        F.col("cep"),
        F.col("latitude"),
        F.col("longitude")
    )
)
display(df_geolocalizacao.limit(5))

In [0]:
df_hospedes = (
    df_hospedes.join(df_geolocalizacao, on="cep", how="left")
)

In [0]:
catalog_table = "production.trusted.tb_hospedes"

if spark.catalog.tableExists(catalog_table):
    delta_table = DeltaTable.forName(spark, catalog_table)
    
    # Merge (upsert) com atualização da update_date
    delta_table.alias("target").merge(
        df_hospedes.alias("source"),
        "target.consumo_id = source.consumo_id"
    ).whenMatchedUpdate(
        set = {
            **{col: f"source.{col}" for col in df_hospedes.columns if col != "update_date"},
            "update_date": "current_date()"
        }
    ).whenNotMatchedInsertAll() \
     .execute()
else:
    # Se não existir, cria a tabela Delta incluindo a update_date
    (
        df_hospedes
        .withColumn("update_date", lit(""))
        .write
        .format("delta")
        .saveAsTable(catalog_table)
    )